In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-manual-new/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

83 experiments


{'commit': '68204a16b6eca1a38accddf1cc2b086d9d893007',
 'id': 'a456a71a344389258ad8165626e7ed62b0e42094',
 'name': 'utile-sida',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.5,
   'system_prompt': 'cot/format-thought.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/cot.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.6066666666666667,
  'f1': 0.7247495282495282,
  'fuzzy_match': 0.7,
  '2hops': {'exact_match': 0.67,
   'f1': 0.8016050061050062,
   'fuzzy_match': 0.79},
  '3hops': {'exact_match': 0.56,
   'f1': 0.6742460317460318,
   'fuzzy_match': 0.65},
  '4hops': {'exact_match': 0.59, 'f1': 0.698397546897547, 'fuzzy_match': 0.66},
  'gen_token_count': {'all': {'count': 300.0,
    'mean': 58.486666666666665,
    'std': 22.606044909353365,
    'min': 24.0,
    '25%': 43.0,
    '50%': 54.0,
    '75%': 69.25,
    'm

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")
    & (df["metrics.f1"] > 0.1)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

52 experiments after preprocessing


In [4]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,manual/ccot.json,0,1,0.0,"(1, 2)"
1,ccot/format-thought.txt,cq.txt,manual/ccot.json,0,1,0.5,"(1, 2)"
2,ccot/format-thought.txt,cq.txt,manual/ccot.json,1,1,0.0,"(1, 2)"
3,ccot/format-thought.txt,cq.txt,manual/ccot.json,1,1,0.5,"(1, 2)"
4,cok/adapted.txt,cq.txt,manual/cok.json,0,1,0.0,"(1, 2)"
5,cok/adapted.txt,cq.txt,manual/cok.json,0,1,0.5,"(1, 2)"
6,cok/adapted.txt,cq.txt,manual/cok.json,1,1,0.0,"(1, 2)"
7,cok/adapted.txt,cq.txt,manual/cok.json,1,1,0.5,"(1, 2)"
8,cot/format-thought.txt,cq.txt,manual/cot.json,0,1,0.0,"(1, 2)"
9,cot/format-thought.txt,cq.txt,manual/cot.json,0,1,0.5,"(1, 2)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cok/adapted.txt
	cot/format-thought.txt
	cte/format-sro-triples-few.txt
	cte/format-triples-ere-few.txt
	cte/format-triples-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	manual/ccot.json
	manual/cok.json
	manual/cot.json
	manual/cte-triples.json
	manual/direct.json
- params.qa.n_shot
	0
	1
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
	0.5
- params.run
	(1, 2)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)